In [20]:
from functools import reduce
import pickle
import pandas as pd
import os
import re
import time

In [21]:
def make_fset(x):
    x1 = x.split(' ')[0]
    x2 = x.split(' ')[1]
    fset = frozenset({x1,x2})
    return(fset)

In [27]:
data_dir = '../ppi_ml/data/featmats/'
lmat_file = 'featmat_allexps_p3c2.pkl'
fmat_files = [f for f in os.listdir(data_dir) if re.match('.*.pkl', f) and f != lmat_file]
outfile = data_dir+'featmat_final'

In [31]:
flist = sorted(fmat_files, reverse=True)

In [36]:
t0 = time.time()
fmat_list = []
for f in flist:
    print(f'Reading {f} ...')
    with open(data_dir+f, 'rb') as handle:
        df = pickle.load(handle)
        df['frozen_pair'] = [make_fset(i) for i in df['ID']]
        df.drop(['ID'], axis=1, inplace=True)
        fmat_list.append(df)
print(f'Total time to read in & format all files: {time.time() - t0} seconds')

Reading featmat_tsar.pkl ...
Reading featmat_plants.pkl ...
Reading featmat_humap.pkl ...
Reading featmat_excavate.pkl ...
Reading featmat_animals.pkl ...
Reading featmat_allconcat.pkl ...
Reading featmap_tsar_norm.pkl ...
Reading featmap_plants_norm.pkl ...
Reading featmap_excavate_norm.pkl ...
Reading featmap_animals_norm.pkl ...
Reading featmap_allconcat_norm.pkl ...
Total time to read in & format all files: 400.05625915527344 seconds


In [37]:
t0 = time.time()
with open(data_dir+lmat_file, 'rb') as handle:
    lmat = pickle.load(handle)
    lmat['frozen_pair'] = [make_fset(i) for i in lmat['ID']]
    for df in fmat_list:
        lmat = lmat.merge(df, how='left', on=['frozen_pair'])
        lmat.fillna(0, inplace=True)
lmat.drop(['frozen_pair'], axis=1, inplace=True)
print(f'Total time to merge featmats: {time.time() - t0} seconds')

Total time to merge featmats: 1993.4076941013336 seconds


In [39]:
lmat.to_pickle(outfile+'.pkl')
lmat.to_csv(outfile, index=False)